In [2]:
# Generating point data for non-landslide locations 

import geopandas as gpd
import pandas as pd
from osgeo import ogr
import random
from shapely.geometry import Point

# Script used to create Non-landslide points 

In [8]:
# landslide points and fishnet

landslides = gpd.read_file('Q:\\Endelige Vektordata\\Landslide_50%roads_removed.shp')
fishnet = gpd.read_file('Q:\\Endelige Vektordata\\Fishnet_40_Training_PartsRemoved.shp')

# External Validation

#landslides = gpd.read_file('Q:\\Endelige Vektordata\\ExVal_Landslide_50%roads_removed.shp')
#fishnet = gpd.read_file('Q:\\Endelige Vektordata\\Fishnet_40_ExternalValidation.shp')

In [9]:
# Buffer the landslide points by 120 meters
buffered_landslides = landslides.buffer(120)

# Exclude grid cells that intersect with buffered landslide points
exclusion_zone = buffered_landslides.unary_union
candidate_cells = fishnet[~fishnet.intersects(exclusion_zone)]

selected_cells = candidate_cells.sample(n=len(landslides))# equal to number of points in 'landslides' point data

# Generate a random point within each selected cell
def generate_random_point_in_cell(cell):
    minx, miny, maxx, maxy = cell.bounds
    while True:
        random_point = Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
        if random_point.within(cell):
            return random_point

non_landslide_points = selected_cells.geometry.apply(generate_random_point_in_cell)

non_landslide_gdf = gpd.GeoDataFrame(geometry=non_landslide_points)
non_landslide_gdf.to_file('Q:\\Endelige Vektordata\\Non_landslides_50%roadsremoved_2.shp', driver='ESRI Shapefile')
print('Non-landslide points generated and saved as Shapefile.')

Non-landslide points generated and saved as Shapefile.


Combining landslide and non-landslide shapefiles

In [10]:
non_landslide = gpd.read_file('Q:\\Endelige Vektordata\\Non_landslides_50%roadsremoved_2.shp')
# populate with the target variable
non_landslide['Landslide'] = 0

non_landslide.to_file('Q:\\Endelige Vektordata\\Non_landslides_50%roadsremoved_2.shp')

In [11]:
landslide = gpd.read_file('Q:\\Endelige Vektordata\\Landslide_50%roadsremoved_w_landslidedata.shp')
#landslide['Landslide'] = 1
#landslide.to_file('Q:\\Endelige Vektordata\\ExVal_Landslide_50%roads_removed.shp')


Combine geodataframes

In [12]:
common_crs = 'epsg:25833'
non_landslide = non_landslide.to_crs(common_crs)
landslide = landslide.to_crs(common_crs)

combined = pd.concat([non_landslide, landslide], ignore_index=True)

combined.to_file('Q:\\Endelige Vektordata\\Landslide_non_landslide_50%roadsremoved_2.shp')

Creating the CSV file for ML training (proceed after data sampling, when the points is populated with values)

In [3]:
trainingdata = gpd.read_file('E:\\LSM - NGU metode\\Landslide_non_landslide_NGU_metode_Training.shp')
trainingdata.to_csv("E:\\LSM - NGU metode\\TrainingData_NGU_method.csv", index=False)

In [4]:
exvalidationdata = gpd.read_file('E:\\LSM - NGU metode\\Landslide_non_landslide_NGU_metode_External.shp')
exvalidationdata.to_csv("E:\\LSM - NGU metode\\ExternalValidation_NGU_method.csv", index=False)